In [10]:
import pandas as pd
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('font', **{'size': 35})
#params = {'axes.labelsize': 48, 'legend.fontsize': 35, 'xtick.labelsize': 35, 'ytick.labelsize': 35}
#matplotlib.rcParams.update(params)
matplotlib.use("pgf")
matplotlib.rcParams.update({
    'axes.labelsize': 100, 'legend.fontsize': 35, 'xtick.labelsize': 35, 'ytick.labelsize': 35,
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [11]:
eng_df = pd.read_csv("eng_final.csv")
perf_df = pd.read_csv("perf_final.csv")

eng_df_group = eng_df.groupby("participant_id")
feedback_shift = 5 # CHECK IT!
engagement_window = 3
plot_users_games = True

sequence_telling_duration_map = {3: 8, 5: 14, 7: 20} # MEASURE IT!
switch_attention_duration = 1

In [12]:
for key in eng_df_group.groups.keys():
    # change to moving window (looking into the future) because timestamp is made before giving the feedback, that is why one can not see the change in engagement after giving the feedback
    eng_qt = eng_df_group.get_group(key)['eng_qt'].ewm(alpha=0.75).mean()
    #eng_tab = eng_df_group.get_group(key)['eng_tab'].ewm(alpha=0.75).mean()
    #eng_qt = eng_df_group.get_group(key)['eng_qt'].rolling(engagement_window, center=True).mean()
    #eng_tab = eng_df_group.get_group(key)['eng_tab'].rolling(engagement_window, center=True).mean()
    eng_df.loc[eng_df['participant_id']==key, 'eng_qt'] = list(eng_qt)
    #eng_df.loc[eng_df['participant_id']==key, 'eng_tab'] = list(eng_tab)

eng_df.dropna(inplace=True)

In [13]:
eng_df

,participant_id,secs,eng_qt
0,03DEQR1O,1647936429,-0.333333
1,03DEQR1O,1647936430,-0.866667
2,03DEQR1O,1647936431,0.250794
3,03DEQR1O,1647936432,0.814902
4,03DEQR1O,1647936433,0.953861
...,...,...,...
9541,Z7U8NLC9,1648041551,-1.000000
9542,Z7U8NLC9,1648041552,-1.000000
9543,Z7U8NLC9,1648041553,-1.000000
9544,Z7U8NLC9,1648041554,-1.000000


In [14]:
pd.read_csv("eng_final.csv")

,participant_id,secs,eng_qt
0,03DEQR1O,1647936429,-0.333333
1,03DEQR1O,1647936430,-1.000000
2,03DEQR1O,1647936431,0.600000
3,03DEQR1O,1647936432,1.000000
4,03DEQR1O,1647936433,1.000000
...,...,...,...
9541,Z7U8NLC9,1648041551,-1.000000
9542,Z7U8NLC9,1648041552,-1.000000
9543,Z7U8NLC9,1648041553,-1.000000
9544,Z7U8NLC9,1648041554,-1.000000


In [15]:
# Measure engagement with the time shift when the feedback is given
perf_df_cpy = perf_df.copy(deep=True)
final_df = pd.merge(left=perf_df_cpy, right=eng_df, on=['participant_id', 'secs'])
final_df['feedback'] = final_df['feedback'].shift(1, fill_value=0)
final_df['start'] = 0
final_df['talking_time'] = final_df['length']
final_df = final_df.replace({'talking_time': sequence_telling_duration_map})

for user in final_df['participant_id'].unique():
    final_df.loc[final_df['participant_id']== user, 'start'] = \
        final_df.loc[final_df['participant_id']== user, 'secs'].shift(1, fill_value=0)
    final_df.loc[(final_df['participant_id']== user) &
                 (final_df['start']==0), 'start'] = \
        int(final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'secs'] - \
        final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'duration'] -\
        final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'talking_time'])

# Average engagement over entire task solving time + time when feedback is given
for timestamp in list(final_df['secs']):
    timestamp = int(timestamp)
    down_lim = int(final_df.loc[final_df['secs']==timestamp, 'start'])
    up_lim = timestamp - int(final_df.loc[final_df['secs']==timestamp, 'duration'])

    final_df.loc[final_df['secs']==timestamp, 'eng_qt'] = eng_df.loc[(eng_df['secs'] >= down_lim) & (eng_df['secs'] <= up_lim), 'eng_qt'].mean()
    # sequence_telling_period = sequence_telling_duration_map[int(final_df.loc[final_df['secs']==timestamp, 'length'])]
    # when feedback is given, engagement from qt should be higher, we account for that
    # if int(final_df.loc[final_df['secs']==timestamp, 'feedback'])>0:
    #      final_df.loc[final_df['secs']==timestamp, 'eng_qt'] = eng_df.loc[
    #          ((eng_df['secs'] >= down_lim - sequence_telling_period - switch_attention_duration) &
    #          (eng_df['secs'] <= down_lim - switch_attention_duration)) |
    #          ((eng_df['secs'] >= timestamp + switch_attention_duration) &
    #           (eng_df['secs'] <= timestamp + switch_attention_duration + feedback_shift)), 'eng_qt'].mean()
    #
    # # else we measure engagement before recreating the sequence
    # else:
    #     final_df.loc[final_df['secs']==timestamp, 'eng_qt'] = eng_df.loc[
    #          (eng_df['secs'] >= down_lim - sequence_telling_period - switch_attention_duration) &
    #          (eng_df['secs'] <= down_lim - switch_attention_duration), 'eng_qt'].mean()

final_df.to_csv("output/final.csv", header=True, index=None, sep=',', float_format='%10.4f', mode='w')

In [16]:
final_df

,participant_id,secs,id,length,correct,feedback,duration,eng_qt,start,talking_time
0,1CZ1CL1P,1648134409,0,7,0,0,17.0509,1.000000,1648134371,20
1,1CZ1CL1P,1648134452,1,7,0,0,12.4892,0.311374,1648134409,20
2,1CZ1CL1P,1648134490,2,5,0,1,6.6176,0.661166,1648134452,14
3,1CZ1CL1P,1648134519,3,5,0,0,5.2145,0.672812,1648134490,14
4,1CZ1CL1P,1648134550,4,3,1,2,6.2580,0.606410,1648134519,8
...,...,...,...,...,...,...,...,...,...,...
195,1PE38CJI,1647955491,5,7,0,0,8.5898,-0.796597,1647955454,20
196,1PE38CJI,1647955532,6,7,0,0,11.3520,-0.651613,1647955491,20
197,1PE38CJI,1647955578,7,5,1,2,13.0154,-0.458824,1647955532,14
198,1PE38CJI,1647955612,8,5,1,0,9.0707,-0.789743,1647955578,14


In [17]:
if plot_users_games:
    for user in eng_df_group.groups.keys():
        qt = eng_df.groupby("participant_id").get_group(user)[['eng_qt', 'secs']]
        time_offset = qt['secs'].to_numpy()[0]
        qt['secs'] = qt['secs']-time_offset

        length = perf_df.groupby("participant_id").get_group(user)[['length', 'secs']]
        length['secs'] = length['secs']-time_offset

        length_3 = length.loc[length['length'] == 3, 'secs']
        length_3 = np.array([length_3, length_3])

        length_5 = length.loc[length['length'] == 5, 'secs']
        length_5 = np.array([length_5, length_5])

        length_7 = length.loc[length['length'] == 7, 'secs']
        length_7 = np.array([length_7, length_7])

        solving_start = perf_df.groupby('participant_id').get_group(user)[['duration', 'secs']].round()
        solving_start['secs'] = solving_start['secs']-time_offset
        solving_start = solving_start['secs'] - solving_start['duration']

        feedback = perf_df.groupby("participant_id").get_group(user)[['feedback', 'secs']]
        feedback['secs'] = feedback['secs']-time_offset
        feedback_en = feedback.loc[feedback['feedback']==1, 'secs']+feedback_shift
        feedback_ch = feedback.loc[feedback['feedback']==2, 'secs']+feedback_shift

        plt.figure(figsize=(20, 4), dpi=100)
        plt.rc('axes', labelsize=100)
        qt_handl, = plt.plot(qt.secs, qt.eng_qt, label="Engagement", color='b')
        #tab_handl, = plt.plot(tab.secs, tab.eng_tab, label="Engagement from tablet", color='c')
        length_3_handl = plt.plot(length_3, np.array([[-1, 1]]*length_3.shape[1]).T, label="End of level 1", color='r', linestyle='-', linewidth=3)
        length_5_handl = plt.plot(length_5, np.array([[-1, 1]]*length_5.shape[1]).T, label="End of level 2", color='r', linestyle='--', linewidth=3)
        length_7_handl = plt.plot(length_7, np.array([[-1, 1]]*length_7.shape[1]).T, label="End of level 3", color='r', linestyle=':', linewidth=3)

        solv_start_handl = plt.scatter(solving_start, [0]*solving_start.shape[0], label="Solving start", color='black', linewidths=20, marker='x')

        feedback_en_handl = plt.scatter(feedback_en, [0]*feedback_en.shape[0], label="Encouraging feedback", color='m', linewidths=20, marker='o')
        feedback_ch_handl = plt.scatter(feedback_ch, [0]*feedback_ch.shape[0], label="Challenging feedback", color='y', linewidths=20, marker='o')

        #plt.legend(handles=[qt_handl, tab_handl, length_3_handl[0], length_5_handl[0], length_7_handl[0], solv_start_handl, feedback_en_handl, feedback_ch_handl])
        plt.legend(handles=[qt_handl, length_3_handl[0], length_5_handl[0], length_7_handl[0], solv_start_handl, feedback_en_handl, feedback_ch_handl], bbox_to_anchor=(1.33, 0.5), loc='center right')

        if user in ['J0YH72SI', '5J7PWO3G']:
            plt.ylabel("Engagement")
            plt.xlabel("Time [s]")
            if user=='J0YH72SI':
                plt.xlim(50, 435)
            elif user == '5J7PWO3G':
                plt.xlim(50, 470)
            plt.grid()
            plt.savefig(f"plots/{user}_sequence_learning.pdf", bbox_inches='tight')
        else:
            plt.ylabel("Engagement")
            plt.xlabel("Time [s]")
            plt.grid()
            plt.title(f"Sequence learning game for user {user}")
            plt.savefig(f"plots/{user}_sequence_learning.pdf", bbox_inches='tight')

/tmp/ipykernel_15653/3367009424.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(20, 4), dpi=100)


In [18]:
# Checking if giving feedback has any effect
comp_df = pd.DataFrame()
comp_df['participant_id'] = final_df["participant_id"].unique()
comp_df['improvement'] = 0

for user in final_df['participant_id'].unique():
    eng_improvements = []
    for level in final_df.loc[(final_df['participant_id']== user) &
                         (final_df['feedback'] >= 1), 'length'].unique():
        eng_improvement = \
            final_df.loc[(final_df['participant_id']== user) &
                         (final_df['feedback'] >= 1) &
                         (final_df['length'] == level), 'eng_qt'].mean() - \
            final_df.loc[(final_df['participant_id']== user) &
                         (final_df['feedback'] == 0) &
                         (final_df['length'] == level), 'eng_qt'].mean()
        eng_improvements.append(1 if eng_improvement>0 else 0)

    eng_improvements = np.array(eng_improvements)
    prob = eng_improvements.sum()/eng_improvements.shape[0]
    comp_df.loc[comp_df['participant_id'] == user, 'improvement'] = prob

comp_df

,participant_id,improvement
0,1CZ1CL1P,0.333333
1,5J7PWO3G,0.666667
2,SYBO5F61,1.000000
3,6RGY40ES,0.666667
4,PTEM0K27,0.333333
5,Q4ABT87L,0.000000
6,J0YH72SI,0.666667
7,M4OE3RP5,0.333333
8,1MBU59SJ,0.000000
9,03DEQR1O,0.333333
